In [ ]:
#%%Encoding categorical variables
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
features = []; categorical_features = []
num_of_columns = x_data.shape[1]
 
for i in range(0, num_of_columns):
 column_name = x_data.columns[i]
 column_type = x_data[column_name].dtypes
 
 if i != num_of_columns: 
  features.append(column_name)
 
 if column_type == 'object':
  le.fit(x_data[column_name])
  feature_classes = list(le.classes_)
  encoded_feature = le.transform(x_data[column_name])
  x_data[column_name] = pd.DataFrame(encoded_feature)
 
  if i != num_of_columns: 
   categorical_features.append(column_name)
x_data.info()
x_data.head()

In [ ]:
#%% Applying XGBoost

#Splitting into training and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0)

import xgboost as xgb
gbm_model = xgb.XGBClassifier(max_depth=5,
                              tree_method = 'exact',
                              n_estimators=500,
                              learning_rate=0.5)

gbm_model.fit(x_train, y_train)

gbm_pred = gbm_model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
confusion_matrix = confusion_matrix(gbm_pred, y_test)
print(confusion_matrix)
print("Accuracy:",accuracy_score(gbm_pred, y_test))
print("Recall:",recall_score(gbm_pred, y_test))
print("Precision:",precision_score(gbm_pred, y_test))
print("F1 Score:",f1_score(gbm_pred, y_test))

In [ ]:
#%% Light GBM

#Splitting training and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.25, random_state = 0)

# Create the LightGBM data containers
import lightgbm as lgb
train_data = lgb.Dataset(x_train, label=y_train, categorical_feature = ['campaign_type','brand_type','category','age_range','marital_status'])
test_data = lgb.Dataset(x_test, label=y_test)

In [ ]:
# Train the model
parameters = {
    'max_depth':10,
    'application':'binary',
    'objective':'binary',
    'metric':'binary_logloss',
    'boosting':'gbdt',
    'num_leaves':31,
    'feature_fraction':0.5,
    'bagging_fraction':0.5,
    'learning_rate':0.5,
    'min_data':50,
    'verbose':0
}
lgb_model = lgb.train(parameters,
                      train_data,
                      valid_sets=test_data,
                      num_boost_round=500,
                      early_stopping_rounds=100)

In [ ]:
#Prediction
y_pred = lgb_model.predict(x_test)

#convert into binary values
for i in range(0,len(y_pred)):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print("Accuracy:",accuracy_score(y_test, y_pred))
print("Recall:",recall_score(y_test, y_pred))
print("Precision:",precision_score(y_test, y_pred))
print("F1 Score:",f1_score(y_test, y_pred))